Acording to the notebook of @gregorycalvez, feature 41-45 are constant for a stock / day combination. https://www.kaggle.com/gregorycalvez/de-anonymization-price-quantity-stocks. 

I tried to retrieve anonymized stock day ID column from these features.

I am not sure if it is real. Any feedbacks are welcome!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import defaultdict
from tqdm.notebook import tqdm
import pickle
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [ ]:
data_dir = "/kaggle/input/jane-street-market-prediction"
train = pd.read_csv(os.path.join(data_dir,'train.csv'))

In [ ]:
print(train.feature_41.isna().sum())
print(train.feature_42.isna().sum())
print(train.feature_43.isna().sum())
print(train.feature_44.isna().sum())
print(train.feature_45.isna().sum())

feature_44 and feature_45 contain NaN.

In [ ]:
train['feature_44'] = train['feature_44'].fillna(0)
train['feature_45'] = train['feature_45'].fillna(0)

In [ ]:
print(len(train.feature_41.unique()))
print(len(train.feature_42.unique()))
print(len(train.feature_43.unique()))
print(len(train.feature_44.unique()))
print(len(train.feature_45.unique()))

In [ ]:
print(train.feature_41.max())
print(train.feature_42.max())
print(train.feature_43.max())
print(train.feature_44.max())
print(train.feature_45.max())
print(train.feature_41.min())
print(train.feature_42.min())
print(train.feature_43.min())
print(train.feature_44.min())
print(train.feature_45.min())

In [ ]:
def add_stock_day_id(df, stock_day_dict, verbose=True):
    max_f=1000
    min_f=-50
    df['stock_day_unique_values']=(df['feature_41']-min_f)*(max_f**0)+(df['feature_42']-min_f)*(max_f**1)+(df['feature_43']-min_f)*(max_f**2)+(df['feature_44']-min_f)*(max_f**3)+(df['feature_45']-min_f)*(max_f**4)
    if len(stock_day_dict) == 0:
        i=1
    else:
        i=max(stock_day_dict.values())+1
    stock_day_id = np.zeros(len(df), dtype=np.int32)
    for cnt, row in (enumerate(tqdm(df[['stock_day_unique_values']].values)) if verbose else enumerate(df[['stock_day_unique_values']].values)):
        stock_day_unique_values = float(row[0])
        if stock_day_dict[stock_day_unique_values] == 0:
            stock_day_id[cnt] = i
            stock_day_dict[stock_day_unique_values]=i
            i = i +1
        else:
            stock_day_id[cnt] = stock_day_dict[stock_day_unique_values]
    df['stock_day_id'] = stock_day_id
    return df

from collections import defaultdict
stock_day_dict = defaultdict(int)
train = add_stock_day_id(train, stock_day_dict)

In [ ]:
print(len(train.stock_day_id.unique()))

In [ ]:
train.head()

# The data of stock_day_id == 1

In [ ]:
train[train.stock_day_id==1]

# Some Plots

In [ ]:
train[:100000].plot.scatter(x = 'stock_day_id', y = 'date')

In [ ]:

train[:100000].plot.scatter(x = 'stock_day_id', y = 'resp')

# Save Files

In [ ]:
def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj, f)


def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data

In [ ]:
pickle_dump(train.stock_day_id.values, 'stock_day_id.pickle')
pickle_dump(stock_day_dict, 'stock_day_dict.pickle')

Do you have any ideas to retrieve stock ID not stock day ID?